In [1]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
import os
import datetime
from datetime import timedelta, date
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [2]:
def get_labels(labels_file_path):
    """
    This method gets channel labels from the labels.dat file from the house directory
    
    Input:
    labels_file_path = Path to the labels.dat file.
    
    Output:
    labels_df = Channel and appliance name dataframe.
    labels_dict = Channel and appliance name dictionary.
    """
    labels_df = pd.read_csv(labels_file_path, sep='\\s+', names=['Channel_id', 'Appliance'])
    labels_df["Channel_id"] = ["channel_"+str(i) for i in range(1,labels_df.shape[0]+1)]
    labels_dict = dict()
    for row in labels_df.iterrows():
        labels_dict[row[1]["Channel_id"]] = row[1]["Appliance"]
    return labels_df, labels_dict

In [3]:
def get_channel_list(house_path):
    """
    This method gets the list of channels from the house directory.
    
    Input:
    house_path = path to the house directory.
    
    Output:
    channel_list = list of all the channels except channel_1 (mains).
    """
    if(house_path[-1] != '/'):
        house_path = house_path + '/'
    channel_list = []
    for item in os.listdir(house_path):
        if 'channel_' in item and item != "channel_1.dat":
            channel_list.append(item[:-4])
    return channel_list

In [4]:
def get_channel_data(on_off_data_dir_path, channel_list):
    """
    This method loads channel on/off data from .npy files 
    generated by running 'Resampling_and_generating_appliance_on_off_data' file 
    
    Input:
    file_path = Path to the .npy files for all the channels
    equipments = List of channels for the house being analyzed
    
    Output:
    equip_dict = Dictionary of equipment and the datetime data when the equipment was turned on
    """
    channel_dict = dict()
    for channel in channel_list:
        if 'channel_' in channel and channel != "channel_1.dat":    
            # print( on_off_data_dir_path + channel)
            channel_data = np.load(on_off_data_dir_path + channel + ".npy")
            channel_data = list(channel_data)
            # Filter out only those instances where appliance is on
            channel_data = [ x for x in channel_data if x!= '0']
            # Populating dictionary with channel data using channel name as key
            channel_dict[channel] = list(channel_data) 
    return channel_dict

In [5]:
def get_min_max_dates(path_to_resampled_channel_data, channel_list):
    """
    This method finds the range of dates in which all the appliances were recorded //
    and returns a min and a max date for given appliances. 
    
    Input:
    path_to_resampled_channel_data = Path to the resampled data directory
    number_of_channels = Total number of channels in the house
    
        
    Output:
    min_date = The earliest date on which an appliance usage was recorded.
    max_date = The latest date on which an appliance usage was recorded.
    """
    min_date = datetime.datetime.max.date()
    max_date = datetime.datetime.min.date()

    for i in reversed(range(2, len(channel_list) + 1)):
        cd = np.load(path_to_resampled_channel_data + "channel_"+str(i)+".npy")
        for item in cd:
            if(item != '0'):
                datetime_obj = datetime.datetime.strptime(item, "%Y-%m-%d %H:%M:%S")
                temp_date = datetime_obj.date()
                if(temp_date < min_date):
                    min_date = temp_date
                break;

        for item in reversed(cd):
            if(item != '0'):
                datetime_obj = datetime.datetime.strptime(item, "%Y-%m-%d %H:%M:%S")
                temp_date = datetime_obj.date()
                if(temp_date > max_date):
                    max_date = temp_date
                break;
    return min_date, max_date

In [6]:
def get_dates_list(min_date, max_date):
    """
    This method finds the dates between a given set of dates.
    
    Input:
    date1 = Start date
    date2 = End date
    
    Output:
    List of date strings in between these dates
    """
    Dates = []
    start_dt = min_date
    end_dt = max_date
    for dt in daterange(start_dt, end_dt):
        Dates.append(dt.strftime("%Y-%m-%d"))
    return Dates

In [7]:
def daterange(date1, date2):
    """
    This method finds the dates between a given set of dates.
    
    Input:
    date1 = Start date
    date2 = End date
    
    Output:
    List of dates in between these dates
    """
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)

In [8]:
def get_all_times_of_day(interval):
    """
    This method generates a list of times of a day seperated by specified interval.
    
    Input:
    interval = The gap between two neighboring time slots
    
    Output:
    Time = List of times seperated by specified interval
    """
    hour = ['00','01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23']
    minute = ['00', str(interval)[:-3]]
    second = '00'
    Time = []
    for hr in hour:
        for min in minute:
            temp = [hr, min, second]
            temp = ':'.join(temp)
            Time.append(temp)
    return Time

In [9]:
def get_usage_data_for_day(channel_data_dict, channel_list, date):
    """
    This method extracts the appliances that are on and their usage data for a particular day. 
    
    Input:
    channel_data_dict = Dictionary of channel and their on/off data.
    channel_list = List of channels for this house
    date = Date for which used appliances and their usage needs to be extracted
    
    Output:
    cleaned_day_data = Dictionary of appliances used that day and their usage (on timings).
    
    """
    day_data = dict()
    for channel in channel_list:
        channel_data = list(channel_data_dict[channel])
        channel_data = [ x for x in channel_data if date in x]
        day_data[channel] = channel_data
    cleaned_day_data = dict()    
    for channel in day_data.keys():
        if len(day_data[channel]) == 0:
            continue
        else:
            cleaned_day_data[channel] = day_data[channel]
    no_of_channel_w = len(cleaned_day_data.keys()) 
#     print("No of channel working on {%s} are :"%(date),end = " ")
#     print(no_of_channel_w)       
    return cleaned_day_data 

In [10]:
def get_usage_data_for_time(day_data, channel_data_dict, date, time, channel_list):
    """
    This method extracts the appliances that are on and their usage data for a particular time of the day. 
    
    Input:
    day_data = Dictionary of channel and their on/off data for the day.
    channel_data_dict = Complete dictionary of channel and their on/off data.
    date = Date for which used appliances and their usage needs to be extracted
    time = Time for which used appliances and their usage needs to be extracted
    channel_list = List of channels for this house
        
    Output:
    list = List of channels on for that time of the day.
    
    """
    temp_list = [date, time]
    temp_time = ' '.join(temp_list)
    time_data = dict()
    time_data[time] = []
    for channel in day_data.keys():
        temp_data = list(channel_data_dict[channel])
        temp_data = [ x for x in temp_data if temp_time in x]
        if len(temp_data)==0:
            continue
        else:
            time_data[channel] = temp_data
#     print("No of channels working at %s %s are :"%(date, time),end =" ")
#     print(len(time_data.keys()))       
    return list(time_data.keys())

In [11]:
def data_extractor(channel_list, channel_data_dict, dates, times):
    """
    This method extracts the appliances that are on and their usage data for a given date range. 
    
    Input:
    channel_list = List of channels for this house.
    channel_data_dict = Complete dictionary of channel and their on/off data.
    dates = Dates for which used appliances and their usage needs to be extracted.
    times = Times for which used appliances and their usage needs to be extracted.
    
        
    Output:
    list = List-of-lists of channels that are On for that period.
    
    """
    transactions = []
    no_of_ch = len(channel_list)
    # Iterate over dates in the date range
    for date in dates:
        # Get appliance usage data for day
        day_data = get_usage_data_for_day(channel_data_dict, channel_list, date)
        # Iterate over times in the 24 hour time range
        for time in times:
            # Get appliances on for time
            temp_list = get_usage_data_for_time(day_data, channel_data_dict, date, time, channel_list)           
            if len(temp_list)==0:
                continue
            else:
                transactions.append(temp_list)
    return transactions           

In [12]:
def divide_data_into_time(times, apriori_data):
    """
    This method seperates ON appliance sequences by time and Saves them in a dictionary with time slices as the key
    
    Input:
    times = List of time slices.
    apriori_data = Output of the data_extractor().
    
    Output:
    Dataframe of support and frequent itemsets.
    """
    time_sequence_dict = dict()
    for timestamp in times:
        sequence_list = []
        for data in apriori_data:
            if timestamp in data:
                sequence_list.append(data)
        time_sequence_dict[timestamp] = sequence_list
    return time_sequence_dict

In [13]:
def get_support_and_itemsets(apriori_data, minimum_support):
    """
    This method runs the apriori algorithm on the input data and returns frequent itemsets with their respective supports
    
    Input:
    apriori_data = List-of-lists of appliances On at a particular time for each day.
    minimum_support = Minimum support for getting the frequent itemsets.
    
    Output:
    Dataframe of support and frequent itemsets
    """
    te = TransactionEncoder()
    data = te.fit(apriori_data).transform(apriori_data)
    data = pd.DataFrame(data, columns = te.columns_)
    return apriori(data, min_support = minimum_support, use_colnames = True)

In [14]:
def get_channels_from_frequent_itemsets(frequent_itemsets_df):
    """
    This method returns the set of unique channels present in the support-frequent itemsets dataframe.
    This method was used to check the appliance coverage.
    
    Input:
    frequent_itemsets_df = Dataframe of support and frequent itemsets.
    
    Output:
    Set of channels present in the entire frequent itemsets
    """
    channels = set()
    for item in frequent_itemsets_df.itemsets:
        for entry in list(item):
            channels.add(entry)
    return channels

In [15]:
def get_appliances_from_channels(list_of_channels, labels_df):
    """
    This method returns a list of appliance names given a list of channels.
    
    Input:
    list_of_channels = List of channels for which names to be extracted.
    labels_df = Dataframe of channel ids and appliance names.
    
    Output:
    List of appliance names corresponding to each channel in list_of_channels
    """
    result = []
    for item in list_of_channels:
        result.append(labels_df.set_index('Channel_id').at[item, 'Appliance'])
    return result

In [16]:
def get_channels_from_rules(rule_df):
    """
    This method returns the set of unique channels present in the time-rules dataframe.
    This method was used to check the appliance coverage.
    
    Input:
    rules_df = Dataframe of time and appliances On.
    
    Output:
    Set of channels present in the rules dataframe
    """
    channel_list = []
    for sequence in rule_df['consequents']:
        list_of_channels = str(sequence)[12:-3].replace("'",'').split(", ")
        for item in list_of_channels:
            if item not in channel_list:
                channel_list.append(item)
    return channel_list

In [17]:
def get_missing_channel_list(rule_df, full_channel_set):
    """
    This method returns the set of unique channels not present in the time-rules dataframe.
    This method was used to check the appliance coverage.
    
    Input:
    rules_df = Dataframe of time and appliances On.
    full_channel_set = Set of complete channels in the house
    
    Output:
    Set of channels not present in the rules dataframe
    """
    channel_set = set()
    for sequence in rule_df['consequents']:
        list_of_channels = sequence[12:-3].replace("'",'').split(", ")
        channel_set.update(list_of_channels)
    result_1 = full_channel_set - channel_set
    return result_1

In [18]:
def change_names(recs, show_name_dict):
    res = []
    for item in recs:
        res.append(show_name_dict[item])
    return res

In [19]:
def get_display_names_dict(labels_to_name_file):
    display_names_dict = dict()
    display_names_df = pd.read_csv(labels_to_name_file, names= ["Labels", "Name"], header = 0)
    for index, row in display_names_df.iterrows():
        display_names_dict[row.Labels] = row.Name
    return display_names_dict

In [20]:
##### CHANGE SHOW_NAME_DICT TO DISPLAY_NAME_DICT

def convert_rule_mining_recs_to_csv(recs_df, show_name_dict):
    recommendation_list = []
    for item in recs_df['Recommendations']:
        recs = []
        item = item[1:-1]
        for channel in item.split(", "):
            channel = channel[2:-2]
            recs.append(channel)
        recommendations = get_appliances_from_channels(recs, labels_df)
        recommendations = change_names(recommendations, show_name_dict)
        recommendation_list.append(",".join(appliance for appliance in recommendations))
    time_recommendation_df = pd.DataFrame({"Time" : Time, "Recommendations" : recommendation_list})
    return time_recommendation_df

### Declare necessary variables

In [33]:
house = 2

print("House : " + str(house))

path_to_house = "../../../../Dataset/ukdale/House_" + str(house) + "/"
labels_file_path = path_to_house + "labels.dat"
labels_to_name_file = "../Labels_to_name_files/House_" + str(house) + ".csv"
channel_status_file_dir = "../Utils/Channel_On_Off_data/House_" + str(house) + "/"
apriori_data_output_file = channel_status_file_dir + "apriori_data.py"
rule_files_dir = "./Rule_Files/House_" + str(house) + "/"
# recommendations_file = "./Recommendations/House_" + str(house) + "/recommendations_" + str(min_support) + "_" + str(min_confidence) + ".csv"
recommendations_file = "./Recommendations/House_" + str(house) + "/Recommendations.csv"

resampling_time_in_min = "30min"
min_support = 0.02
min_confidence = 0.02
considered_rules = 200
channel_list = get_channel_list(path_to_house)

print("path_to_house : " + path_to_house)
print("labels_file : " + labels_file_path)
print("labels_to_name_file : " + labels_to_name_file)
print("channel_status_file_dir : " + channel_status_file_dir)
print("apriori_data_output_file : " + apriori_data_output_file)
print("rule_files_dir : " + rule_files_dir)
print("recommendations_file : " + recommendations_file)

print("resampling_time_in_min : " + resampling_time_in_min)
print("min_support : " + str(min_support))
print("min_confidence : " + str(min_confidence))
print("considered_rules : " + str(considered_rules))

House : 2
path_to_house : ../../../../Dataset/ukdale/House_2/
labels_file : ../../../../Dataset/ukdale/House_2/labels.dat
labels_to_name_file : ../Labels_to_name_files/House_2.csv
channel_status_file_dir : ../Utils/Channel_On_Off_data/House_2/
apriori_data_output_file : ../Utils/Channel_On_Off_data/House_2/apriori_data.py
rule_files_dir : ./Rule_Files/House_2/
recommendations_file : ./Recommendations/House_2/Recommendations.csv
resampling_time_in_min : 30min
min_support : 0.02
min_confidence : 0.02
considered_rules : 200


In [22]:
show_name_dict = get_display_names_dict(labels_to_name_file)

In [23]:
# display_names_dict

### Get labels from the labels.dat file

In [24]:
labels_df, labels_map = get_labels(labels_file_path)

In [25]:
labels_df

,Channel_id,Appliance
0,channel_1,aggregate
1,channel_2,laptop
2,channel_3,monitor
3,channel_4,speakers
4,channel_5,server
5,channel_6,router
6,channel_7,server_hdd
7,channel_8,kettle
8,channel_9,rice_cooker
9,channel_10,running_machine


### Get house related data

In [26]:
# Get channel list and data from channel files
channel_list = get_channel_list(channel_status_file_dir)
channel_data = get_channel_data(channel_status_file_dir, channel_list)

# Get max and min date in the channel data
min_date, max_date = get_min_max_dates(channel_status_file_dir, channel_list)

# Generate date and time list 
Dates = get_dates_list(min_date, max_date)
Time = get_all_times_of_day(resampling_time_in_min)

### Extract appliance usage data from the channels

In [27]:
# # Generate appliance ON data for each time slice of each day and save it.
apriori_dt = data_extractor(channel_list, channel_data, Dates, Time)
np.save(apriori_data_output_file, apriori_dt)

In [28]:
# len(apriori_dt)

### Generate rules for each time of the day 

In [29]:
# %%time

# Generate frequent itemsets and rules one time slice at a time and save them in a dictionary. 

time_itemset_map = dict()
time_rules_map = dict()
time_channels_map_from_itemsets = dict()
time_channels_map_from_rules = dict()

time_appliance_map = divide_data_into_time(Time, apriori_dt)

for timestamp in list(time_appliance_map.keys()):
    
    print("Generating Itemsets for : " + str(timestamp))
    # Generate frequent itemsets
    time_itemset_map[timestamp] = get_support_and_itemsets(time_appliance_map[timestamp], min_support)
    
#     print("Generating Rules for : " + str(timestamp))
    # Generate rules
    time_rules_map[timestamp] = association_rules(time_itemset_map[timestamp], metric="confidence", min_threshold = min_confidence)
#     print(time_rules_map[timestamp].shape)
    
#     print("Filtering Rules for : " + str(timestamp))
    # Filter rules which starts from current time slice
    rules_df = time_rules_map[timestamp]
    time_rules_map[timestamp] = rules_df[rules_df['antecedents'] == frozenset({timestamp})]
    
    # Get channels from frequent itemsets
    time_channels_map_from_itemsets[timestamp] = get_channels_from_frequent_itemsets(time_itemset_map[timestamp])
    
    # Get channels from rules
    time_channels_map_from_rules[timestamp] = get_channels_from_rules(time_rules_map[timestamp])


Generating Itemsets for : 00:00:00
Generating Itemsets for : 00:30:00
Generating Itemsets for : 01:00:00
Generating Itemsets for : 01:30:00
Generating Itemsets for : 02:00:00
Generating Itemsets for : 02:30:00
Generating Itemsets for : 03:00:00
Generating Itemsets for : 03:30:00
Generating Itemsets for : 04:00:00
Generating Itemsets for : 04:30:00
Generating Itemsets for : 05:00:00
Generating Itemsets for : 05:30:00
Generating Itemsets for : 06:00:00
Generating Itemsets for : 06:30:00
Generating Itemsets for : 07:00:00
Generating Itemsets for : 07:30:00
Generating Itemsets for : 08:00:00
Generating Itemsets for : 08:30:00
Generating Itemsets for : 09:00:00
Generating Itemsets for : 09:30:00
Generating Itemsets for : 10:00:00
Generating Itemsets for : 10:30:00
Generating Itemsets for : 11:00:00
Generating Itemsets for : 11:30:00
Generating Itemsets for : 12:00:00
Generating Itemsets for : 12:30:00
Generating Itemsets for : 13:00:00
Generating Itemsets for : 13:30:00
Generating Itemsets 

### Save the rules in CSV files to a rules directory

In [30]:
for timestamp, df in time_rules_map.items():
    df.to_csv(rule_files_dir + timestamp + ".csv", header=True, index=False)

### Generate Recommendations and save them to a dataframe

In [32]:
recommendation_list = []
for timestamp in Time:
    rule_df = time_rules_map[timestamp].sort_values(by=['confidence'], ascending=False)
    recommended_channels = get_channels_from_rules(rule_df[:considered_rules])
    recommendations = get_appliances_from_channels(recommended_channels, labels_df)
    recommendations = change_names(recommendations, show_name_dict)
    recommendation_list.append(",".join(appliance for appliance in recommendations))
time_recommendation_df = pd.DataFrame({"Time" : Time, "Recommendations" : recommendation_list})

In [34]:
time_recommendation_df.to_csv(recommendations_file)

In [35]:
# apriori_dt